In [1]:
import sys
sys.path.append("../")

In [2]:
import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import json
import math
import numpy as np
import os
import pandas as pd
from dask import compute, delayed
from dask.dataframe import DataFrame
from dask.distributed import Client, LocalCluster, wait, worker_client
from vani.core.analysis import Analysis
from vani.core.metrics import filter_asymptote_delayed, filter_delayed, flatten_delayed, merge_delayed, sort_delayed
from vani.utils.file_utils import ensure_dir
from vani.utils.json_encoders import NpEncoder
from vani.utils.logger import create_logger, format_log


In [3]:
client = Client(LocalCluster(n_workers=16, local_directory=f"/var/tmp/playground-quartz/local"))
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 48,Total memory: 125.50 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41662,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 48
Started: Just now,Total memory: 125.50 GiB
Comm: tcp://127.0.0.1:42394,Total threads: 3
Dashboard: http://127.0.0.1:33701/status,Memory: 7.84 GiB
Nanny: tcp://127.0.0.1:38276,


In [4]:
CACHE_DIR = "cached"
METRICS_DIR = "metrics"
INDEX_DIR = "indexed"


def compute_metrics_file_id(ddf: DataFrame, fg_index: str, log_dir: str):
    unique_filenames_d = unique_filenames_delayed(ddf=ddf, log_dir=log_dir,
                                                  dask_key_name=f"unique-filenames-{fg_index}")
    save_filenames_d = save_filenames_delayed(filenames=unique_filenames_d, log_dir=log_dir,
                                              dask_key_name=f"save-filenames-{fg_index}")
    metrics_d = metrics_filenames_delayed(ddf=ddf, filenames=unique_filenames_d,
                                          dask_key_name=f"metrics-{fg_index}")
    return [unique_filenames_d, save_filenames_d, [metrics_d]]


def load_global_min_max(log_dir: str):
    with open(f"{log_dir}/global.json") as file:
        global_min_max = json.load(file)
    return global_min_max


@delayed
def metrics_filenames_delayed(ddf: DataFrame, filenames: list, fg_index='file_id'):
    print('calculating metrics for filenames', len(filenames))
    tasks_d = []
    for filename in filenames:
        target_ddf_d = Analysis.target_ddf_delayed(ddf=ddf, start=filename, stop=filename,
                                                   dask_key_name=f"target-ddf-{fg_index}-{filename}")
        filter_d = filter_delayed(ddf=target_ddf_d, fg_index=fg_index, start=filename, stop=filename,
                                  dask_key_name=f"filter-{fg_index}-{filename}")
        flatten_d = flatten_delayed(filter_d, dask_key_name=f"flatten-{fg_index}-{filename}")
        tasks_d.append(flatten_d)
    print('num of tasks created', len(tasks_d))
    # with worker_client() as client:
    #     print('submitting on', client)
    #     metrics_f = client.compute(tasks_d)
    #     print('gathering tasks', len(metrics_f))
    #     metrics = client.gather(metrics_f)
    metrics = dask.compute(tasks_d)
    print('computed metrics', type(metrics))
    print('computed metrics', len(metrics))
    # print('computed metrics', metrics)
    return metrics


def read_and_index_logs(prefix: str, fg_index: str, log_dir: str, use_cache=True):
    dask_suffix = f"{prefix}-{fg_index}"
    fg_index_dir = f"{log_dir}/{INDEX_DIR}/{fg_index}"
    if use_cache and os.path.exists(f"{fg_index_dir}/_metadata"):
        ddf_d = read_parquet_delayed(log_dir=fg_index_dir, index=[fg_index],
                                     dask_key_name=f"read-parquet-{dask_suffix}")
        persisted_ddf_d = persist_ddf_delayed(ddf=ddf_d, dask_key_name=f"persist-ddf-{dask_suffix}")
        return [ddf_d, persisted_ddf_d]
    ddf_d = read_parquet_delayed(log_dir=log_dir, dask_key_name=f"read-parquet-{dask_suffix}")
    indexed_ddf_d = set_ddf_index_delayed(ddf=ddf_d, fg_index=fg_index,
                                          dask_key_name=f"set-index-{dask_suffix}")
    persisted_ddf_d = persist_ddf_delayed(ddf=indexed_ddf_d, dask_key_name=f"persist-ddf-{dask_suffix}")
    partitioned_ddf_d = repartition_delayed(ddf=persisted_ddf_d,
                                            dask_key_name=f"repartition-ddf-{dask_suffix}")
    save_ddf_d = save_ddf_delayed(ddf=partitioned_ddf_d, log_dir=log_dir, fg_index=fg_index,
                                  dask_key_name=f"save-ddf-{dask_suffix}")
    return [ddf_d, indexed_ddf_d, persisted_ddf_d, save_ddf_d, partitioned_ddf_d]


@delayed
def read_parquet_delayed(log_dir: str, index: list = None):
    if index:
        print("Index specified", index)
        return dd.read_parquet(f"{log_dir}/*.parquet", calculate_divisions=True, index=index)
    print("Index not specified")
    return dd.read_parquet(f"{log_dir}/*.parquet", index=False)


@delayed
def repartition_delayed(ddf: DataFrame, partition_size='128MB'):
    return ddf.repartition(partition_size=partition_size)


@delayed
def persist_ddf_delayed(ddf: DataFrame):
    ddf = ddf.persist()
    wait(ddf)
    return ddf


@delayed
def save_ddf_delayed(ddf: DataFrame, log_dir: str, fg_index: str):
    ddf.to_parquet(f"{log_dir}/{INDEX_DIR}/{fg_index}")


@delayed
def save_filenames_delayed(filenames: list, log_dir: str):
    filenames = list(filenames)
    filenames.sort()
    with open(f"{log_dir}/filenames.json", "w") as file:
        json.dump(filenames, file, cls=NpEncoder)


@delayed
def set_ddf_index_delayed(ddf: DataFrame, fg_index: str):
    return ddf.set_index([fg_index])


@delayed
def unique_filenames_delayed(ddf: DataFrame, log_dir: str):
    if os.path.exists(f"{log_dir}/filenames.json"):
        with open(f"{log_dir}/filenames.json", "r") as file:
            unique_filenames = json.load(file)
    else:
        unique_filenames = ddf.index.unique().compute()
    return unique_filenames


In [5]:
%%time

cm1_dir = "/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet"

cm1_tmid_ddf_d = read_and_index_logs('cm1', 'tmid', log_dir=cm1_dir)
cm1_tmid_ddf = compute(*cm1_tmid_ddf_d)[-1]
cm1_tmid_ddf

Index specified ['tmid']
CPU times: user 539 ms, sys: 289 ms, total: 829 ms
Wall time: 1.98 s


,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,acc_pat,bandwidth,duration,file_id,proc_id
npartitions=1,,,,,,,,,,,,,,,,,,,
4967534,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,int32,float32,float32,int64,int64
6683055727,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
cm1_tmid_ddf.index.min().compute(), cm1_tmid_ddf.index.max().compute()

(4967534, 6683055727)

In [7]:
%%time
num_time_tasks = 2 ** 10
cm1_time_max = cm1_tmid_ddf.index.max().compute()
cm1_time_min = cm1_tmid_ddf.index.min().compute()
cm1_time_interval = math.ceil(cm1_time_max * 1.0 / num_time_tasks)
cm1_time_range = np.arange(0, cm1_time_max, cm1_time_interval)
cm1_time_range
# pd.cut(cm1_tmid_ddf['tmid'], cm1_time_range)

CPU times: user 34.2 ms, sys: 10.9 ms, total: 45 ms
Wall time: 68.1 ms


array([         0,    6526422,   13052844, ..., 6663476862, 6670003284,
       6676529706])

In [8]:
%%time

# .groupby([pd.cut(cm1_tmid_ddf.reset_index()['tmid'], cm1_time_range), 'io_cat']) \

cm1_time_agg_ddf = cm1_tmid_ddf \
    .groupby([cm1_tmid_ddf.index.name, 'proc_id', 'io_cat']) \
    .agg({'duration': sum, 'size': sum}) \
    .compute() \
    .groupby([cm1_tmid_ddf.index.name, 'io_cat']) \
    .agg({'duration': max, 'size': sum})

cm1_time_agg_ddf

CPU times: user 67.7 ms, sys: 15.2 ms, total: 82.9 ms
Wall time: 109 ms


,,duration,size
tmid,io_cat,,
4967534,3,0.000003,0
5003463,3,0.000007,0
5004308,3,0.000122,0
5005592,3,0.000001,0
5013357,1,0.001470,16777216
...,...,...,...
6683029938,3,0.000066,0
6683041824,2,0.002297,8454144
6683053729,3,0.000066,0


In [9]:
%%time
cm1_proc_ddf_d = read_and_index_logs('cm1', 'proc_id', log_dir=cm1_dir)
cm1_proc_ddf = compute(*cm1_proc_ddf_d)[-1]
cm1_proc_ddf

Index specified ['proc_id']
CPU times: user 195 ms, sys: 56.8 ms, total: 252 ms
Wall time: 1.38 s


,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,acc_pat,bandwidth,duration,tmid,file_id
npartitions=1,,,,,,,,,,,,,,,,,,,
6860131432589748884,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,int32,float32,float32,int64,int64
6860131432673569428,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
%%time

cm1_proc_agg_ddf = cm1_proc_ddf \
    .groupby([cm1_proc_ddf.index.name, 'proc_id', 'io_cat']) \
    .agg({'duration': max, 'size': sum}) \
    .compute() \
    .groupby([cm1_proc_ddf.index.name, 'io_cat']) \
    .agg({'duration': sum, 'size': sum})

cm1_proc_agg_ddf

CPU times: user 45.5 ms, sys: 4.17 ms, total: 49.7 ms
Wall time: 63.9 ms


duration        size
proc_id             io_cat                      
6860131432589748884 1       0.001307    50331648
                    2       0.006606  1230852944
                    3       1.970968    17674980
6860131432589814420 1       0.000009    16777216
                    3       0.000013           0
...                              ...         ...
6860131432673438356 3       0.000013           0
6860131432673503892 1       0.000008    16777216
                    3       0.000012           0
6860131432673569428 1       0.000010    16777216
                    3       0.000014           0

[2561 rows x 2 columns]

In [11]:
cm1_proc_agg_ddf.groupby(level=0).sum().sort_values('duration', ascending=False)

,duration,size
proc_id,,
6860131432629070484,2.191853,16777216
6860131432615963284,2.191401,16777216
6860131432594991764,2.167567,16777216
6860131432613341844,2.165946,16777216
6860131432652663444,2.165902,16777216
...,...,...
6860131432648206996,0.000019,16777216
6860131432632150676,0.000019,16777216
6860131432632937108,0.000019,16777216


In [12]:
cm1_proc_ddf.loc[6860131432629070484] \
    .groupby([cm1_proc_ddf.index.name, 'proc_id', 'io_cat']) \
    .agg({'duration': max, 'size': sum}) \
    .compute() \
    .groupby([cm1_proc_ddf.index.name, 'io_cat']) \
    .agg({'duration': sum, 'size': sum})

duration      size
proc_id             io_cat                    
6860131432629070484 1       0.000010  16777216
                    3       2.191843         0

In [13]:
%%time
cm1_file_ddf_d = read_and_index_logs('cm1', 'file_id', log_dir=cm1_dir)
cm1_file_ddf = compute(*cm1_file_ddf_d)[-1]
cm1_file_ddf

Index specified ['file_id']
CPU times: user 202 ms, sys: 45.2 ms, total: 248 ms
Wall time: 1.36 s


,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,acc_pat,bandwidth,duration,tmid,proc_id
npartitions=1,,,,,,,,,,,,,,,,,,,
901974072163427748,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,int32,float32,float32,int64,int64
4531734248727221027,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [14]:
%%time

cm1_file_agg_ddf = cm1_file_ddf \
    .groupby([cm1_file_ddf.index.name, 'proc_id', 'io_cat']) \
    .agg({'duration': sum, 'size': sum}) \
    .compute() \
    .groupby([cm1_file_ddf.index.name, 'io_cat']) \
    .agg({'duration': max, 'size': sum})

cm1_file_agg_ddf

CPU times: user 45.7 ms, sys: 8.68 ms, total: 54.4 ms
Wall time: 67 ms


duration   size
file_id             io_cat                 
901974072163427748  3       0.000004      0
4531734244435428412 2       0.000067  47144
                    3       0.000648      0
4531734244436508198 2       0.000078  49340
                    3       0.000881      0
...                              ...    ...
4531734248720417485 3       0.001617      0
4531734248722472348 2       0.000101  49340
                    3       0.000823      0
4531734248727221027 2       0.000110  49340
                    3       0.001923      0

[1549 rows x 2 columns]

In [15]:
cm1_file_ddf.map_partitions(lambda df: df.groupby([df.index.name, 'proc_id', 'io_cat']).agg(
    {'duration': sum, 'size': sum}).groupby([cm1_file_ddf.index.name, 'io_cat']).agg({'duration': max, 'size': sum})).compute()
# cm1_file_ddf.npartitions


duration   size
file_id             io_cat                 
901974072163427748  3       0.000004      0
4531734244435428412 2       0.000067  47144
                    3       0.000648      0
4531734244436508198 2       0.000078  49340
                    3       0.000881      0
...                              ...    ...
4531734248720417485 3       0.001617      0
4531734248722472348 2       0.000101  49340
                    3       0.000823      0
4531734248727221027 2       0.000110  49340
                    3       0.001923      0

[1549 rows x 2 columns]

In [16]:
%%time
dd.from_pandas(cm1_file_agg_ddf.reset_index(), npartitions=cm1_file_ddf.npartitions).to_parquet(f"{cm1_dir}/{METRICS_DIR}/file_id")

CPU times: user 109 ms, sys: 22.4 ms, total: 131 ms
Wall time: 610 ms


[None]

In [17]:
%%time
dd.read_parquet(f"{cm1_dir}/{METRICS_DIR}/file_id/*.parquet", index=['file_id', 'io_cat']).compute()

CPU times: user 31.5 ms, sys: 10.5 ms, total: 42 ms
Wall time: 49.2 ms


duration   size
file_id             io_cat                 
901974072163427748  3       0.000004      0
4531734244435428412 2       0.000067  47144
                    3       0.000648      0
4531734244436508198 2       0.000078  49340
                    3       0.000881      0
...                              ...    ...
4531734248720417485 3       0.001617      0
4531734248722472348 2       0.000101  49340
                    3       0.000823      0
4531734248727221027 2       0.000110  49340
                    3       0.001923      0

[1549 rows x 2 columns]

In [18]:
cm1_file_agg_ddf.groupby(level=0).sum().sort_values('duration', ascending=False)

,duration,size
file_id,,
4531734245058167199,2.220052,1996330
4531734247156679037,2.198721,44564480
4531734244909082846,0.325141,49340
4531734247515089012,0.246595,44564480
4531734246187791667,0.236232,46923776
...,...,...
4531734247840317221,0.000297,47144
4531734245613589701,0.000292,47144
4531734247359713448,0.000291,47144


In [19]:
cm1_file_agg_ddf.loc[(4531734247156679037,)]

,duration,size
io_cat,,
2,0.006853,44564480
3,2.191869,0


In [20]:
cm1_file_ddf.loc[4531734247156679037]['filename'].unique().compute()[0]

'/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_files/les_ConvBoundLayer/cm1out_000001_s.dat'

In [21]:
cm1_file_ddf.loc[4531734247156679037] \
    .groupby([cm1_file_ddf.index.name, 'proc_id', 'io_cat']) \
    .agg({'duration': sum, 'size': sum}) \
    .compute() 


duration      size
file_id             proc_id             io_cat                    
4531734247156679037 6860131432589748884 2       0.006853  44564480
                                        3       0.000600         0
                    6860131432592370324 3       1.939265         0
                    6860131432594991764 3       2.167586         0
                    6860131432597613204 3       2.129485         0
                    6860131432600234644 3       2.069088         0
                    6860131432602856084 3       2.151718         0
                    6860131432605477524 3       2.054140         0
                    6860131432608098964 3       2.139141         0
                    6860131432610720404 3       2.054162         0
                    6860131432613341844 3       2.165964         0
                    6860131432615963284 3       2.191415         0
                    6860131432618584724 3       1.992256         0
                    6860131432621206164 3       2.114205         0
                    6860131432623827604 3       2.164693         0
                    6860131432626449044 3       2.164794         0
                    6860131432629070484 3       2.191869         0
                    6860131432631691924 3       2.099497         0
                    6860131432634313364 3       1.970189         0
                    6860131432636934804 3       2.164304         0
                    6860131432639556244 3       1.952912         0
                    6860131432642177684 3       1.994167         0
                    6860131432644799124 3       2.054758         0
                    6860131432647420564 3       2.164233         0
                    6860131432650042004 3       2.010971         0
                    6860131432652663444 3       2.164746         0
                    6860131432655284884 3       2.151749         0
                    6860131432657906324 3       1.952589         0
                    6860131432660527764 3       2.116171         0
                    6860131432663149204 3       2.116318         0
                    6860131432665770644 3       2.026828         0
                    6860131432668392084 3       2.025364         0
                    6860131432671013524 3       2.151073         0

In [22]:
hacc_dir = "/p/vast1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet"

hacc_tmid_ddf_d = read_and_index_logs('hacc', 'tmid', log_dir=hacc_dir)
hacc_tmid_ddf = compute(*hacc_tmid_ddf_d)[-1]
hacc_tmid_ddf

Index specified ['tmid']


,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,acc_pat,bandwidth,duration,file_id,proc_id
npartitions=1,,,,,,,,,,,,,,,,,,,
9600220,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,int32,float32,float32,int64,int64
337020894,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [23]:
%%time

hacc_time_agg_ddf = hacc_tmid_ddf \
    .groupby([hacc_tmid_ddf.index, 'proc_id', 'io_cat']) \
    .agg({'duration': sum, 'size': sum}) \
    .compute() \
    .groupby(['tmid', 'io_cat']) \
    .agg({'duration': max, 'size': sum})

hacc_time_agg_ddf

CPU times: user 146 ms, sys: 39.8 ms, total: 185 ms
Wall time: 308 ms


,,duration,size
tmid,io_cat,,
9600220,0,5.900000e-06,0
9600410,0,0.000000e+00,0
9602780,0,6.200000e-06,0
9602965,0,9.000000e-07,0
9604790,0,6.200000e-06,0
...,...,...,...
336997176,0,9.449000e-04,0
336997196,0,9.489000e-04,0
337006621,0,9.060000e-04,0


In [24]:
hacc_file_ddf_d = read_and_index_logs('hacc', 'file_id', log_dir=hacc_dir)
hacc_file_ddf = compute(*hacc_file_ddf_d)[-1]
hacc_file_ddf

Index specified ['file_id']


,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,acc_pat,bandwidth,duration,tmid,proc_id
npartitions=1,,,,,,,,,,,,,,,,,,,
3341432780458102534,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,int32,float32,float32,int64,int64
6142509188972423790,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [25]:
%%time

hacc_file_agg_ddf = hacc_file_ddf \
    .groupby([hacc_file_ddf.index, 'proc_id', 'io_cat']) \
    .agg({'duration': sum, 'size': sum}) \
    .compute() \
    .groupby(['file_id', 'io_cat']) \
    .agg({'duration': max, 'size': sum})

hacc_file_agg_ddf

CPU times: user 56.2 ms, sys: 7.92 ms, total: 64.1 ms
Wall time: 87.3 ms


duration        size
file_id             io_cat                       
3341432780458102534 3        0.000330           0
3341432780459322875 3        0.000079           0
3341432780460151857 3        0.000111           0
3341432780463386647 3        0.000073           0
3341432780467942539 3        0.000051           0
...                               ...         ...
5347710268971829974 3        3.753991  5419040768
5347710268972954261 1        0.588363   662700032
                    2        1.918945   662700032
                    3        3.843605  5419040768
6142509188972423790 0       14.476766           0

[5122 rows x 2 columns]

In [26]:
hacc_file_agg_ddf.groupby(level=0).sum().sort_values('duration', ascending=False)

,duration,size
file_id,,
6142509188972423790,14.476766,0
5347710267150565560,10.367187,6744440832
5347710266789296543,10.321506,6744440832
5347710265503319523,10.317016,6744440832
5347710268612693268,10.296165,6744440832
...,...,...
3341432784126556057,0.000020,0
3341432781123988900,0.000020,0
3341432783630115954,0.000019,0


In [27]:
hacc_file_agg_ddf.loc[(5347710267150565560,)]

,duration,size
io_cat,,
1,1.259819,662700032
2,0.658486,662700032
3,8.448881,5419040768


In [28]:
hacc_file_ddf.loc[5347710267150565560]['filename'].unique().compute()[0]

'/p/gpfs1/iopp/temp/hacc_dir/test-Part00000085-of-00001280.data'

In [29]:
hacc_proc_ddf_d = read_and_index_logs('hacc', 'proc_id', log_dir=hacc_dir)
hacc_proc_ddf = compute(*hacc_proc_ddf_d)[-1]
hacc_proc_ddf

Index specified ['proc_id']


,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,acc_pat,bandwidth,duration,tmid,file_id
npartitions=1,,,,,,,,,,,,,,,,,,,
5117778030848165332,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,int32,float32,float32,int64,int64
5117778030931985876,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [30]:
%%time

hacc_proc_agg_ddf = hacc_proc_ddf \
    .groupby([hacc_proc_ddf.index, 'proc_id', 'io_cat']) \
    .agg({'duration': max, 'size': sum}) \
    .compute() \
    .groupby(['proc_id', 'io_cat']) \
    .agg({'duration': sum, 'size': sum})

hacc_proc_agg_ddf

CPU times: user 45.8 ms, sys: 13 ms, total: 58.8 ms
Wall time: 81.5 ms


duration        size
proc_id             io_cat                      
5117778030848165332 0       3.946803           0
                    1       0.196155   662700032
                    2       0.600341   662700032
                    3       3.855495  5419040768
5117778030848230868 0       3.938079           0
...                              ...         ...
5117778030931920340 3       0.002239  5419040768
5117778030931985876 0       6.651499           0
                    1       0.147614   662700032
                    2       0.264437   662700032
                    3       0.004232  5419040768

[5120 rows x 2 columns]

In [31]:
hacc_proc_agg_ddf.groupby(level=0).sum().sort_values('duration', ascending=False)

,duration,size
proc_id,,
5117778030853735892,18.607897,6744440832
5117778030854260180,18.429443,6744440832
5117778030853539284,18.240847,6744440832
5117778030855898580,18.103481,6744440832
5117778030860092884,18.099543,6744440832
...,...,...
5117778030918747604,5.215243,6744440832
5117778030918682068,5.169654,6744440832
5117778030918092244,5.166821,6744440832


In [32]:
hacc_proc_agg_ddf.loc[(5117778030853735892,)]

,duration,size
io_cat,,
0,8.865714,0
1,0.815058,662700032
2,0.487713,662700032
3,8.439411,5419040768


In [33]:
hacc_proc_ddf.loc[5117778030853735892] \
    .groupby([hacc_proc_ddf.index.name, 'proc_id', 'io_cat']) \
    .agg({'duration': max, 'size': sum}) \
    .compute() \
    .groupby([hacc_proc_ddf.index.name, 'io_cat']) \
    .agg({'duration': sum, 'size': sum})


duration        size
proc_id             io_cat                      
5117778030853735892 0       8.865714           0
                    1       0.815058   662700032
                    2       0.487713   662700032
                    3       8.439411  5419040768

In [34]:
%%time

mon_dir = "/p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet"

mon_tmid_ddf_d = read_and_index_logs('montage', 'tmid', log_dir=mon_dir)
mon_tmid_ddf = compute(*mon_tmid_ddf_d)[-1]
mon_tmid_ddf

Index specified ['tmid']
CPU times: user 1.35 s, sys: 291 ms, total: 1.64 s
Wall time: 9.57 s


,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,acc_pat,bandwidth,duration,file_id,proc_id
npartitions=21,,,,,,,,,,,,,,,,,,,
106000,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,int32,float32,float32,int64,int64
3733001,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171108138,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752590343,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [35]:
%%time

mon_time_agg_ddf = mon_tmid_ddf \
    .groupby([mon_tmid_ddf.index.name, 'proc_id', 'io_cat']) \
    .agg({'duration': max, 'size': sum}) \
    .compute() \
    .groupby([mon_tmid_ddf.index.name, 'io_cat']) \
    .agg({'duration': sum, 'size': sum})

mon_time_agg_ddf

CPU times: user 6.04 s, sys: 2.45 s, total: 8.49 s
Wall time: 16.1 s


,,duration,size
tmid,io_cat,,
106000,3,0.000010,0
107724,3,0.000011,0
107798,3,0.000182,0
109577,3,0.000190,0
110153,3,0.000017,0
...,...,...,...
576913696,3,0.000029,0
1752182840,3,0.001715,0
1752194245,2,0.000410,3534398


In [36]:
%%time
mon_file_ddf_d = read_and_index_logs('montage', 'file_id', log_dir=mon_dir)
mon_file_ddf = compute(*mon_file_ddf_d)[-1]
mon_file_ddf

Index specified ['file_id']
CPU times: user 1.18 s, sys: 247 ms, total: 1.43 s
Wall time: 8.43 s


,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,acc_pat,bandwidth,duration,tmid,proc_id
npartitions=20,,,,,,,,,,,,,,,,,,,
-5561148475055268842,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,int32,float32,float32,int64,int64
1020634716318353501,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707285563540423880,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8946970060350424228,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [37]:
%%time

mon_file_agg_ddf = mon_file_ddf \
    .groupby([mon_file_ddf.index.name, 'proc_id', 'io_cat']) \
    .agg({'duration': sum, 'size': sum}) \
    .compute() \
    .groupby([mon_file_ddf.index.name, 'io_cat']) \
    .agg({'duration': max, 'size': sum})

mon_file_agg_ddf

CPU times: user 99.8 ms, sys: 14.1 ms, total: 114 ms
Wall time: 172 ms


duration     size
file_id              io_cat                   
-5561148475055268842 3       0.000006        0
 300058771974416469  1       0.000021     5762
                     2       0.004227  4429440
                     3       0.000505        0
 300058772040730509  1       0.000022     5762
...                               ...      ...
 8946970060347188895 2       0.004402  4619520
                     3       0.000380        0
 8946970060350424228 1       0.000021     5762
                     2       0.004467  4625280
                     3       0.000549        0

[4708 rows x 2 columns]

In [38]:
mon_file_agg_ddf2 = mon_file_ddf.map_partitions(lambda df: df.groupby([df.index.name, 'proc_id', 'io_cat']).agg(
    {'duration': sum, 'size': sum}).groupby([mon_file_ddf.index.name, 'io_cat']).agg({'duration': max, 'size': sum})).compute()
mon_file_agg_ddf2

duration     size
file_id              io_cat                   
-5561148475055268842 3       0.000006        0
 300058771974416469  1       0.000021     5762
                     2       0.004227  4429440
                     3       0.000505        0
 300058772040730509  1       0.000022     5762
...                               ...      ...
 8946970060347188895 2       0.004402  4619520
                     3       0.000380        0
 8946970060350424228 1       0.000021     5762
                     2       0.004467  4625280
                     3       0.000549        0

[4723 rows x 2 columns]

In [39]:
mon_file_agg_ddf.groupby(level=0).sum().sort_values('duration', ascending=False)

,duration,size
file_id,,
7701937538982921416,3.690958,2829513124
7639450094153155784,1.873162,9542800389
6142509188972423790,1.867907,0
7707004088563713224,1.089136,938124444
7703344913866474696,1.019091,946564732
...,...,...
8459451269980390646,0.000009,0
8459710870688662774,0.000009,0
8459472471164147827,0.000009,0


In [40]:
mon_file_agg_ddf2.groupby(level=0).sum().sort_values('duration', ascending=False)

,duration,size
file_id,,
7701937538982921416,3.690958,2829513124
7639450094153155784,1.873162,9542800389
6142509188972423790,1.867907,0
7707004088563713224,1.089136,938124444
7703344913866474696,1.019091,946564732
...,...,...
8459451269980390646,0.000009,0
8459710870688662774,0.000009,0
8459472471164147827,0.000009,0


In [41]:
mon_file_ddf.loc[7701937538982921416] \
    .groupby(['proc_id', 'io_cat']) \
    .agg({'duration': sum}) \
    .compute() \
    .sort_values('duration', ascending=False)


,,duration
proc_id,io_cat,
9060729382066249156,1,3.582343
5012278936682092996,2,0.108024
5012278936681896388,2,0.098144
5012278936681961924,2,0.089839
5012278936682027460,2,0.084172
5012278936682092996,3,0.000590
5012278936682027460,3,0.000435
5012278936681961924,3,0.000376
9060729382066249156,3,0.000239


In [42]:
mon_file_ddf.loc[7701937538982921416] \
    .groupby([mon_file_ddf.index.name, 'proc_id', 'io_cat']) \
    .agg({'duration': sum, 'size': sum}) \
    .compute() \
    .groupby([mon_file_ddf.index.name, 'io_cat']) \
    .agg({'duration': max, 'size': sum})


duration        size
file_id             io_cat                      
7701937538982921416 1       3.582343  2735911684
                    2       0.108024    93601440
                    3       0.000590           0

In [43]:
%%time
mon_proc_ddf_d = read_and_index_logs('montage', 'proc_id', log_dir=mon_dir)
mon_proc_ddf = compute(*mon_proc_ddf_d)[-1]
mon_proc_ddf

Index specified ['proc_id']
CPU times: user 1.22 s, sys: 260 ms, total: 1.48 s
Wall time: 8.76 s


,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,acc_pat,bandwidth,duration,tmid,file_id
npartitions=20,,,,,,,,,,,,,,,,,,,
115744974875708724,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,int32,float32,float32,int64,int64
145425196050865604,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9060727573885011860,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9060729382066249156,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [44]:
%%time

mon_proc_agg_ddf = mon_proc_ddf \
    .groupby([mon_proc_ddf.index.name, 'proc_id', 'io_cat']) \
    .agg({'duration': max, 'size': sum}) \
    .compute() \
    .groupby([mon_proc_ddf.index.name, 'io_cat']) \
    .agg({'duration': sum, 'size': sum})

mon_proc_agg_ddf

CPU times: user 99.1 ms, sys: 17.2 ms, total: 116 ms
Wall time: 171 ms


duration        size
proc_id             io_cat                      
115744974875708724  1       0.000025    24012853
                    2       0.000011         515
                    3       0.001184         -64
115745060775054644  2       0.000008          84
                    3       0.004452        1932
...                              ...         ...
9060728299736259012 2       0.003434     9653600
                    3       0.000381        1002
9060729382066249156 1       1.362043  2735911684
                    2       0.000385     3332093
                    3       0.001957           0

[853 rows x 2 columns]

In [45]:
mon_proc_agg_ddf.groupby(level=0).sum().sort_values('duration', ascending=False)

,duration,size
proc_id,,
5670209836996549060,1.671605,17785828
5670209836996614596,1.670639,10562788
5670209836996483524,1.633206,17117672
5670209836996417988,1.487604,13149024
9060729382066249156,1.364385,2739243777
...,...,...
1705357240648650036,0.000225,2012
1458149265372138804,0.000218,2016
1029740493937888564,0.000216,2012


In [46]:
mon_proc_agg_ddf.loc[(9060729382066249156,)]

,duration,size
io_cat,,
1,1.362043,2735911684
2,0.000385,3332093
3,0.001957,0


In [47]:
mon_proc_ddf.loc[9060729382066249156] \
    .groupby([mon_proc_ddf.index.name, 'proc_id', 'io_cat']) \
    .agg({'duration': max, 'size': sum}) \
    .compute() \
    .groupby([mon_proc_ddf.index.name, 'io_cat']) \
    .agg({'duration': sum, 'size': sum})

duration        size
proc_id             io_cat                      
9060729382066249156 1       1.362043  2735911684
                    2       0.000385     3332093
                    3       0.001957           0

In [48]:
mon_proc_ddf.loc[9060729382066249156].groupby(['proc_id', 'io_cat']).agg({'duration':max}).compute()

duration
proc_id             io_cat          
9060729382066249156 1       1.362043
                    2       0.000385
                    3       0.001957